# Peak Finding in xarray DataArrays

There are no existing libraries for the purpose, and reducing n-dimensional tensors down to 1D arrays for the scipy peak finding algorithm is subjective as the user has to define what order to unfold the tensor.

In [ ]:
%reload_ext autoreload
%autoreload 3 --print

import xarray as xr
from pca_analysis.get_sample_data import get_zhang_data
from pca_analysis import xr_signal

xr.set_options(display_expand_data=False)

raw_data: xr.DataArray = get_zhang_data()
raw_data.name = "raw_data"


This tensor has 3 modes. We want to detect every peak at every point. There are some libraries for tensor flow, but generally speaking I'm sure that a 1D approach is best. For example


In [ ]:
raw_data


In [ ]:
# from pca_analysis import xr_signal
from pca_analysis.xr_signal import find_peaks_array

ds = raw_data.to_dataset()
test_ds = ds.sel(mz=46).isel(sample=slice(5, 10))

res = test_ds["raw_data"].pipe(find_peaks_array, grouper=["sample", "mz"], x_key="time")

test_ds = test_ds.merge(res)
test_ds


In [ ]:
fig = test_ds.pipe(
    xr_signal.facet_plot_multiple_traces,
    data_keys=["raw_data", "peaks"],
    grouper=["sample", "mz"],
    col_wrap=2,
    x_key="time",
    fig_kwargs=dict(
        y_title="au",
    ),
)
fig.update_layout(height=1500).show()


As we can see, peak finding is successful. It would be useful to add the viz function as an optional output of the peak finding function - given an xarray input, find the peaks and return as a dataset, providing a viz if specified.

In [ ]:
x_key = "time"
array_key = "raw_data"
grouper = ["sample", "mz"]
new_arr_name = "peaks"
test_ds = test_ds.pipe(
    xr_signal.find_peaks_dataset,
    array_key=array_key,
    grouper=grouper,
    x_key=x_key,
    new_arr_name=new_arr_name,
)
test_ds.pipe(
    xr_signal.facet_plot_multiple_traces,
    grouper=grouper,
    x_key=x_key,
    data_keys=[array_key, new_arr_name],
    trace_kwargs=[dict(mode="lines"), dict(mode="markers")],
    col_wrap=3,
)


And as we can see the convenience function works as expected, producing the same result as above.

## Conclusion

We have established a foundation of peak finding on xarray DataArrays and Dataset objects, including result calculation and visualisation. We have also established a convenience function for working at the Dataset level
rather than DataArray.